In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

data = {
    'Arabic': ['مرحبا', 'صباح الخير', 'شكرا', 'أنا آسف', 'أنا احبك'],
    'English': ['Hello', 'Good Morning', 'Thank You', 'Sorry', 'I Love You'],
    'French': ['Salut', 'Bonjour', 'Merci', 'Désolé', 'Je t\'aime']
}

df = pd.DataFrame(data)
df

,Arabic,English,French
0,مرحبا,Hello,Salut
1,صباح الخير,Good Morning,Bonjour
2,شكرا,Thank You,Merci
3,أنا آسف,Sorry,Désolé
4,أنا احبك,I Love You,Je t'aime


In [3]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Define stop words for each language
stop_words_arabic = set(stopwords.words('arabic'))
stop_words_english = set(stopwords.words('english'))
stop_words_french = set(stopwords.words('french'))
import string
string.punctuation
import string

# Function to remove punctuation
def remove_punctuation(text: str):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Apply the function to remove punctuation from each column
df['Arabic'] = df['Arabic'].apply(remove_punctuation)
df['English'] = df['English'].apply(remove_punctuation)
df['French'] = df['French'].apply(remove_punctuation)

In [5]:
# Preprocessing: Lowercase and remove stop words
def remove_stopwords(text, lang):
    if lang == 'arabic':
        return ' '.join([word for word in text.split() if word not in stop_words_arabic])
    elif lang == 'english':
        return ' '.join([word for word in text.split() if word.lower() not in stop_words_english])
    elif lang == 'french':
        return ' '.join([word for word in text.split() if word.lower() not in stop_words_french])
    return text

df['Arabic'] = df['Arabic'].str.lower().apply(lambda x: remove_stopwords(x, 'arabic'))
df['English'] = df['English'].str.lower().apply(lambda x: remove_stopwords(x, 'english'))
df['French'] = df['French'].str.lower().apply(lambda x: remove_stopwords(x, 'french'))

In [6]:
# Add start and end tokens to target languages
df['English'] = df['English'].apply(lambda x: '<start> ' + x + ' <end>')
df['French'] = df['French'].apply(lambda x: '<start> ' + x + ' <end>')

In [7]:
df['English']

0           <start> hello <end>
1    <start> good morning <end>
2           <start> thank <end>
3           <start> sorry <end>
4            <start> love <end>
Name: English, dtype: object

In [8]:
# Tokenization
tokenizer_src = Tokenizer()
tokenizer_src.fit_on_texts(df['Arabic'])
source_sequences = tokenizer_src.texts_to_sequences(df['Arabic'])

tokenizer_tgt_en = Tokenizer()
tokenizer_tgt_en.fit_on_texts(df['English'])
target_sequences_en = tokenizer_tgt_en.texts_to_sequences(df['English'])

tokenizer_tgt_fr = Tokenizer()
tokenizer_tgt_fr.fit_on_texts(df['French'])
target_sequences_fr = tokenizer_tgt_fr.texts_to_sequences(df['French'])

In [9]:
# Padding sequences
max_length = max(max(len(seq) for seq in source_sequences),
                 max(len(seq) for seq in target_sequences_en),
                 max(len(seq) for seq in target_sequences_fr))

source_sequences = pad_sequences(source_sequences, maxlen=max_length, padding='pre')
target_sequences_en = pad_sequences(target_sequences_en, maxlen=max_length, padding='pre')
target_sequences_fr = pad_sequences(target_sequences_fr, maxlen=max_length, padding='pre')

In [10]:
# Train-test split
X_train_src_to_en, X_test_src_to_en, y_train_src_to_en, y_test_src_to_en = train_test_split(source_sequences, target_sequences_en, test_size=0.2, random_state=42)
X_train_en_to_src,X_test_en_to_src,y_train_en_to_src,y_test_en_to_src=train_test_split(target_sequences_en,source_sequences,test_size=0.2,random_state=42)
X_train_src_to_fr, X_test_src_to_fr, y_train_src_to_fr, y_test_src_to_fr = train_test_split(source_sequences, target_sequences_fr, test_size=0.2, random_state=42)
X_train_fr_to_src,X_test_fr_to_src,y_train_fr_to_src,y_test_fr_to_src=train_test_split(target_sequences_fr,source_sequences,test_size=0.2,random_state=42)
X_train_fr_to_en,X_test_fr_to_en,y_train_fr_to_en,y_test_fr_to_en =train_test_split(target_sequences_fr,target_sequences_en,test_size=0.2,random_state=42)
X_train_en_to_fr,X_test_en_to_fr,y_train_en_to_fr,y_test_en_to_fr=train_test_split(target_sequences_en,target_sequences_fr,test_size=0.2,random_state=42)

In [11]:
# Define Encoder
class Encoder(Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(enc_units, return_sequences=True, return_state=True)

    def call(self, x):
        x = self.embedding(x)
        enc_output, state_h, state_c = self.lstm(x)
        return enc_output, state_h, state_c

In [12]:
# Define Decoder
class Decoder(Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(dec_units, return_sequences=True, return_state=True)
        self.fc = Dense(vocab_size, activation='softmax')

    def call(self, x, enc_output, state_h, state_c):
        x = self.embedding(x)
        dec_output, dec_state_h, dec_state_c = self.lstm(x, initial_state=[state_h, state_c])
        output = self.fc(dec_output)
        return output, dec_state_h, dec_state_c

In [13]:
# Hyperparameters
vocab_size_src    = len(tokenizer_src.word_index) + 1
vocab_size_tgt_en = len(tokenizer_tgt_en.word_index) + 1
vocab_size_tgt_fr = len(tokenizer_tgt_fr.word_index) + 1
embedding_dim = 256
units = 512

In [14]:
# Instantiate models
encoder_src = Encoder(vocab_size_src, embedding_dim, units)
encoder_en = Encoder(vocab_size_tgt_en,embedding_dim,units)
encoder_fr = Encoder(vocab_size_tgt_fr,embedding_dim,units)
decoder_src=Decoder(vocab_size_src,embedding_dim,units)
decoder_en = Decoder(vocab_size_tgt_en, embedding_dim, units)
decoder_fr = Decoder(vocab_size_tgt_fr, embedding_dim, units)

In [15]:
# Compile the models
encoder_src.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
encoder_en.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
encoder_fr.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
decoder_src.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
decoder_en.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
decoder_fr.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [16]:
# Prepare the target data for training 
# Shift the target sequences for teacher forcing
y_train_src_to_en_input = y_train_src_to_en[:, :-1]  # All but the last word
y_train_src_to_en_output = y_train_src_to_en[:, 1:]    # All but the first word
y_train_en_to_src_input=y_train_en_to_src[:,:-1]  # All but the last word
y_train_en_to_src_output=y_train_en_to_src[:,1:]  # All but the first word
y_train_src_to_fr_input = y_train_src_to_fr[:, :-1]  # All but the last word
y_train_src_to_fr_output = y_train_src_to_fr[:, 1:]    # All but the first word
y_train_fr_to_src_input=y_train_fr_to_src[:,:-1]  # All but the last word
y_train_fr_to_src_output=y_train_fr_to_src[:,1:]  # All but the first word
y_train_en_to_fr_input= y_train_en_to_fr[:,:-1]  # All but the last word
y_train_en_to_fr_output=y_train_en_to_fr[:,1:]  # All but the first word
y_train_fr_to_en_input=y_train_fr_to_en[:,:-1]  # All but the last word
y_train_fr_to_en_output=y_train_fr_to_en[:,1:]   # All but the first word

In [17]:
y_train_src_to_en_input

array([[0, 1, 8],
       [0, 1, 6],
       [0, 1, 3],
       [0, 1, 7]])

In [18]:
y_train_src_to_en

array([[0, 1, 8, 2],
       [0, 1, 6, 2],
       [0, 1, 3, 2],
       [0, 1, 7, 2]])

In [19]:
# Define a function to train the model
def train_model(encoder, decoder, X, y_input, y_output, epochs=100):
    for epoch in range(epochs):
        for i in range(len(X)):
            # Get the input and output for the current sample
            encoder_input = np.array([X[i]])
            decoder_input = np.array([y_input[i]])

            # Initialize the hidden state of the encoder
            enc_output, state_h, state_c = encoder(encoder_input)

            # Initialize the decoder input and states
            with tf.GradientTape() as tape:
                decoder_output, _, _ = decoder(decoder_input, enc_output, state_h, state_c)

                # Calculate the loss (using sparse categorical crossentropy)
                # Reshape decoder_output to match y_output[i]'s shape
                # The change is in the next line: using decoder_output[0]
                loss = tf.keras.losses.sparse_categorical_crossentropy(y_output[i], decoder_output[0])

            # Backpropagation
            gradients = tape.gradient(loss, decoder.trainable_variables)
            decoder.optimizer.apply_gradients(zip(gradients, decoder.trainable_variables))

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {np.mean(loss.numpy())}')

In [20]:
# Train the English decoder
print("Training Arabic to English Decoder...")
train_model(encoder_src, decoder_en, X_train_src_to_en, y_train_src_to_en_input, y_train_src_to_en_output, epochs=20)

Training Arabic to English Decoder...
Epoch 1/20, Loss: 2.135546922683716
Epoch 2/20, Loss: 1.9705390930175781
Epoch 3/20, Loss: 1.6780797243118286
Epoch 4/20, Loss: 1.4397263526916504
Epoch 5/20, Loss: 1.3516579866409302
Epoch 6/20, Loss: 1.1341012716293335
Epoch 7/20, Loss: 0.9341280460357666
Epoch 8/20, Loss: 0.7627288699150085
Epoch 9/20, Loss: 0.6189457178115845
Epoch 10/20, Loss: 0.5438634157180786
Epoch 11/20, Loss: 0.49920961260795593
Epoch 12/20, Loss: 0.47243019938468933
Epoch 13/20, Loss: 0.4616376459598541
Epoch 14/20, Loss: 0.46247777342796326
Epoch 15/20, Loss: 0.46829068660736084
Epoch 16/20, Loss: 0.47297847270965576
Epoch 17/20, Loss: 0.4733855724334717
Epoch 18/20, Loss: 0.4699072539806366
Epoch 19/20, Loss: 0.46495839953422546
Epoch 20/20, Loss: 0.46083369851112366


In [21]:
print("Training  English to Arabic Decoder...")
train_model(encoder_en, decoder_src, X_train_en_to_src, y_train_en_to_src_input, y_train_en_to_src_output, epochs=20)

Training  English to Arabic Decoder...
Epoch 1/20, Loss: 1.7277189493179321
Epoch 2/20, Loss: 1.6028165817260742
Epoch 3/20, Loss: 1.4762330055236816
Epoch 4/20, Loss: 1.3389692306518555
Epoch 5/20, Loss: 1.1505546569824219
Epoch 6/20, Loss: 0.9338092803955078
Epoch 7/20, Loss: 0.7266533970832825
Epoch 8/20, Loss: 0.5697762966156006
Epoch 9/20, Loss: 0.5232869982719421
Epoch 10/20, Loss: 0.5225786566734314
Epoch 11/20, Loss: 0.5133489370346069
Epoch 12/20, Loss: 0.4995051622390747
Epoch 13/20, Loss: 0.4935086667537689
Epoch 14/20, Loss: 0.4936009645462036
Epoch 15/20, Loss: 0.4928872585296631
Epoch 16/20, Loss: 0.4894077777862549
Epoch 17/20, Loss: 0.4853297770023346
Epoch 18/20, Loss: 0.4826402962207794
Epoch 19/20, Loss: 0.4814740717411041
Epoch 20/20, Loss: 0.4808904230594635


In [22]:
# Train the French decoder
print("Training Arabic to French Decoder...")
train_model(encoder_src, decoder_fr, X_train_src_to_fr, y_train_src_to_fr_input, y_train_src_to_fr_output, epochs=20)

Training Arabic to French Decoder...
Epoch 1/20, Loss: 2.0185165405273438
Epoch 2/20, Loss: 1.8590484857559204
Epoch 3/20, Loss: 1.5889023542404175
Epoch 4/20, Loss: 1.388357162475586
Epoch 5/20, Loss: 1.3023297786712646
Epoch 6/20, Loss: 1.0974689722061157
Epoch 7/20, Loss: 0.9193122386932373
Epoch 8/20, Loss: 0.7572290301322937
Epoch 9/20, Loss: 0.6245836615562439
Epoch 10/20, Loss: 0.5523881912231445
Epoch 11/20, Loss: 0.5054731369018555
Epoch 12/20, Loss: 0.47573187947273254
Epoch 13/20, Loss: 0.46284064650535583
Epoch 14/20, Loss: 0.46258974075317383
Epoch 15/20, Loss: 0.46810850501060486
Epoch 16/20, Loss: 0.4731982946395874
Epoch 17/20, Loss: 0.4743584394454956
Epoch 18/20, Loss: 0.47142815589904785
Epoch 19/20, Loss: 0.46653202176094055
Epoch 20/20, Loss: 0.4621165990829468


In [23]:
print(" Training French to Arabic Decoder")
train_model(encoder_fr,decoder_src,X_train_fr_to_src,y_train_fr_to_src_input,y_train_fr_to_src_output,epochs=20)

 Training French to Arabic Decoder
Epoch 1/20, Loss: 0.48563122749328613
Epoch 2/20, Loss: 0.48548802733421326
Epoch 3/20, Loss: 0.48573175072669983
Epoch 4/20, Loss: 0.48564672470092773
Epoch 5/20, Loss: 0.48502257466316223
Epoch 6/20, Loss: 0.4840656518936157
Epoch 7/20, Loss: 0.4831100404262543
Epoch 8/20, Loss: 0.4823780953884125
Epoch 9/20, Loss: 0.481898695230484
Epoch 10/20, Loss: 0.4815733730792999
Epoch 11/20, Loss: 0.4812770187854767
Epoch 12/20, Loss: 0.4809330701828003
Epoch 13/20, Loss: 0.48053303360939026
Epoch 14/20, Loss: 0.4801109731197357
Epoch 15/20, Loss: 0.4797076880931854
Epoch 16/20, Loss: 0.4793466627597809
Epoch 17/20, Loss: 0.4790273606777191
Epoch 18/20, Loss: 0.47873637080192566
Epoch 19/20, Loss: 0.478456974029541
Epoch 20/20, Loss: 0.47818002104759216


In [24]:
print("Training French to English Decoder")
train_model(encoder_fr,decoder_en,X_train_fr_to_en,y_train_fr_to_en_input,y_train_fr_to_en_output,epochs=20)

Training French to English Decoder
Epoch 1/20, Loss: 0.47043928503990173
Epoch 2/20, Loss: 0.47077512741088867
Epoch 3/20, Loss: 0.47341135144233704
Epoch 4/20, Loss: 0.47628292441368103
Epoch 5/20, Loss: 0.47798100113868713
Epoch 6/20, Loss: 0.4780600070953369
Epoch 7/20, Loss: 0.4768620431423187
Epoch 8/20, Loss: 0.47508731484413147
Epoch 9/20, Loss: 0.47339990735054016
Epoch 10/20, Loss: 0.47219112515449524
Epoch 11/20, Loss: 0.4715368449687958
Epoch 12/20, Loss: 0.47128668427467346
Epoch 13/20, Loss: 0.4712033271789551
Epoch 14/20, Loss: 0.4710737466812134
Epoch 15/20, Loss: 0.4707850217819214
Epoch 16/20, Loss: 0.4703216552734375
Epoch 17/20, Loss: 0.4697379767894745
Epoch 18/20, Loss: 0.46911129355430603
Epoch 19/20, Loss: 0.46850743889808655
Epoch 20/20, Loss: 0.4679598808288574


In [25]:
print("Training English to French Decoder")
train_model(encoder_en,decoder_fr,X_train_en_to_fr,y_train_en_to_fr_input,y_train_en_to_fr_output,epochs=20)

Training English to French Decoder
Epoch 1/20, Loss: 0.47094595432281494
Epoch 2/20, Loss: 0.4714568853378296
Epoch 3/20, Loss: 0.47424137592315674
Epoch 4/20, Loss: 0.4772104024887085
Epoch 5/20, Loss: 0.47894248366355896
Epoch 6/20, Loss: 0.47899118065834045
Epoch 7/20, Loss: 0.4777046740055084
Epoch 8/20, Loss: 0.475801944732666
Epoch 9/20, Loss: 0.47397705912590027
Epoch 10/20, Loss: 0.47264906764030457
Epoch 11/20, Loss: 0.47191622853279114
Epoch 12/20, Loss: 0.47163423895835876
Epoch 13/20, Loss: 0.471556693315506
Epoch 14/20, Loss: 0.47145554423332214
Epoch 15/20, Loss: 0.4711962044239044
Epoch 16/20, Loss: 0.4707476794719696
Epoch 17/20, Loss: 0.4701574146747589
Epoch 18/20, Loss: 0.4695064127445221
Epoch 19/20, Loss: 0.4688683748245239
Epoch 20/20, Loss: 0.46829017996788025


In [26]:
# Evaluate the models on the test set (optional)
def evaluate_model(encoder, decoder, X, y_input, y_output):
    total_loss = 0
    for i in range(len(X)):
        encoder_input = np.array([X[i]])
        decoder_input = np.array([y_input[i]])

        enc_output, state_h, state_c = encoder(encoder_input)

        # Calculate the loss
        decoder_output, _, _ = decoder(decoder_input, enc_output, state_h, state_c)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_output[i], decoder_output[0])
        total_loss += np.mean(loss.numpy())

    print(f'Test Loss: {total_loss / len(X)}')

In [27]:
# Evaluate the English decoder
print("Evaluating Arabic to English Decoder...")
evaluate_model(encoder_src, decoder_en, X_test_src_to_en, y_test_src_to_en[:, :-1], y_test_src_to_en[:, 1:])

Evaluating Arabic to English Decoder...
Test Loss: 2.257819175720215


In [28]:
print("Evaluating English to Arabic Decoder")
evaluate_model(encoder_en,decoder_src,X_test_en_to_src,y_test_en_to_src[:,:-1],y_test_en_to_src[:,1:])

Evaluating English to Arabic Decoder
Test Loss: 2.9857609272003174


In [29]:
# Evaluate the French decoder
print("Evaluating Arabic to French Decoder...")
evaluate_model(encoder_src, decoder_fr, X_test_src_to_fr, y_test_src_to_fr[:, :-1], y_test_src_to_fr[:, 1:])

Evaluating Arabic to French Decoder...
Test Loss: 3.037219285964966


In [30]:
print("Evaluating French to Arabic Decoder")
evaluate_model(encoder_fr,decoder_src,X_test_fr_to_src,y_test_fr_to_src[:,:-1],y_test_fr_to_src[:,1:])

Evaluating French to Arabic Decoder
Test Loss: 2.9436838626861572


In [31]:
print("Evaluating French to English Decoder")
evaluate_model(encoder_fr,decoder_en,X_test_fr_to_en,y_test_fr_to_en[:,:-1],y_test_fr_to_en[:,1:])

Evaluating French to English Decoder
Test Loss: 1.972615122795105


In [32]:
print("Evaluating English to French Decoder")
evaluate_model(encoder_en,decoder_fr,X_test_en_to_fr,y_test_en_to_fr[:,:-1],y_test_en_to_fr[:,1:])

Evaluating English to French Decoder
Test Loss: 2.8439056873321533


In [33]:
# Define the inference function
def translate_sentence(sentence, encoder, decoder, tokenizer_tgt):
    # Preprocess the input sentence
    sequence = tokenizer_src.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Get the encoder output
    enc_output, state_h, state_c = encoder(padded_sequence)

    # Start decoding
    # Check if '<start>' is in word_index, if not, use 1 (often the default for unknown)
    start_token_index = tokenizer_tgt.word_index.get('<start>', 1)
    target_seq = np.array([[start_token_index]])  # Start token
    translated_sentence = []

    for _ in range(max_length):
        output, state_h, state_c = decoder(target_seq, enc_output, state_h, state_c)
        predicted_word_index = np.argmax(output[0, -1, :])
        predicted_word = tokenizer_tgt.index_word.get(predicted_word_index, '')

        if predicted_word == '<end>':  # End token
            break

        translated_sentence.append(predicted_word)
        target_seq = np.array([[predicted_word_index]])

    return ' '.join(translated_sentence)

In [34]:
# # Example usage
# arabic_sentence = 'مرحبا'
# print("Translating Arabic to English:")
# translated_en = translate_sentence(arabic_sentence, encoder, decoder_en, tokenizer_tgt_en)
# print(f"Translation (English): {translated_en}")

# print("Translating Arabic to French:")
# translated_fr = translate_sentence(arabic_sentence, encoder, decoder_fr, tokenizer_tgt_fr)
# print(f"Translation (French): {translated_fr}")

In [35]:
# Save the models
# encoder.save('encoder_model.h5')
# decoder_en.save('decoder_en_model.h5')
# decoder_fr.save('decoder_fr_model.h5')

# Load the models (if needed)
# encoder = tf.keras.models.load_model('encoder_model.h5', custom_objects={'Encoder': Encoder})
# decoder_en = tf.keras.models.load_model('decoder_en_model.h5', custom_objects={'Decoder': Decoder})
# decoder_fr = tf.keras.models.load_model('decoder_fr_model.h5', custom_objects={'Decoder': Decoder})

In [36]:
def translate_user_input():
    user_sentence = input("Enter a sentence in Arabic, English, or French: ").strip().lower()
    target_language = input("Translate to (English/French/Arabic): ").strip().lower()

    # Determine source and target tokenizers
    if target_language == 'english':
        tokenizer_tgt = tokenizer_tgt_en
        decoder = decoder_en
    elif target_language == 'french':
        tokenizer_tgt = tokenizer_tgt_fr
        decoder = decoder_fr
    elif target_language == 'arabic':
        tokenizer_tgt = tokenizer_src
        decoder = decoder_src
    else:
        print("Invalid target language. Choose from English, French, or Arabic.")
        return

    # Detect source language and set encoder
    if all(word in tokenizer_src.word_index for word in user_sentence.split()):
        print("Detected source language: Arabic")
        encoder = encoder_src
    elif all(word in tokenizer_tgt_en.word_index for word in user_sentence.split()):
        print("Detected source language: English")
        encoder = encoder_en
    elif all(word in tokenizer_tgt_fr.word_index for word in user_sentence.split()):
        print("Detected source language: French")
        encoder = encoder_fr
    else:
        print("Unsupported or mixed language. Please try again.")
        return

    # Translate sentence
    translated_sentence = translate_sentence(user_sentence, encoder, decoder, tokenizer_tgt)
    print(f"Translated Sentence ({target_language.capitalize()}): {translated_sentence}")


In [37]:
# Define Reverse Decoder for translating from English/French to Arabic
class ReverseDecoder(Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(ReverseDecoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = LSTM(dec_units, return_sequences=True, return_state=True)
        self.fc = Dense(vocab_size, activation='softmax')

    def call(self, x, enc_output, state_h, state_c):
        x = self.embedding(x)
        dec_output, dec_state_h, dec_state_c = self.lstm(x, initial_state=[state_h, state_c])
        output = self.fc(dec_output)
        return output, dec_state_h, dec_state_c

In [ ]:
# Example usage
translate_user_input()